In [ ]:
!pip install --upgrade google-api-python-client
!pip3 install --upgrade oauth2client

In [ ]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
!pip install connect
!pip install functions
import connect

In [3]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = 'XXXXXXXX' #add your view ID

In [4]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
service = build('analyticsreporting', 'v4', credentials=credentials)

In [54]:
response = service.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:sessions'}],
          'dimensions': [{'name': 'ga:medium'}]
        }]
      }
  ).execute()

In [55]:
dim = []
metric = []
  
for report in response.get('reports', []):
  
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])
  
    for row in rows:
  
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])
  
        for header, dimension in zip(dimensionHeaders, dimensions):
            dim.append(dimension)
  
        for i, values in enumerate(dateRangeValues):
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                metric.append(int(value))

In [56]:
import pandas as pd

In [ ]:
df = pd.DataFrame() 
df["Sessions"]= metric
df["Medium"]= dim
df= df[["Medium","Sessions"]]

print(df)

In [ ]:
df.head()

## Filtering Metrics

In [50]:

response = service.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:pageviews'}],
          'dimensions': [{'name': 'ga:pagePath'}],
          

          "metricFilterClauses": [{
              "filters": [{
                  "metricName": "ga:pageviews",
                  "operator": "GREATER_THAN",
                  "comparisonValue": "2"
          }]
      }]
  }]
}
).execute()

## Filtering Dimensions

In [36]:
response = service.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
          'metrics': [{'expression': 'ga:pageviews'}],
          "dimensions": [{"name": "ga:browser"}, {"name": "ga:country"}],
          "dimensionFilterClauses": [
        {
          "filters": [
            {
              "dimensionName": "ga:browser",
              "operator": "EXACT",
              "expressions": ["Chrome"]
            }
          ]
        }
      ]
    }
  ]
}
).execute()

## Expressions

In [39]:
response = service.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
          "metrics":
      [
        {
          "expression": "ga:goal1completions/ga:users",
          "alias": "completions per user"
        }
      ]
    }
  ]
}
).execute()

## Histograms

In [40]:
response = service.reports().batchGet(
    body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
          "metrics": [{"expression": "ga:sessions"}],
          "dimensions": [
        {
              "name": "ga:sessionCount",
              "histogramBuckets": ["1","10","100","200","400"]
        }
      ],
         "orderBys": [
        {
              "fieldName": "ga:sessionCount",
              "orderType": "HISTOGRAM_BUCKET"
        }
      ]
    }
  ]
}
).execute()

In [42]:
#create two empty lists that will hold our dimentions and sessions data
dim = []
metric = []
  
#Extract Data
for report in response.get('reports', []):
  
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
    rows = report.get('data', {}).get('rows', [])
  
    for row in rows:
  
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])
  
        for header, dimension in zip(dimensionHeaders, dimensions):
            dim.append(dimension)
  
        for i, values in enumerate(dateRangeValues):
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                metric.append(int(value))

In [ ]:
df = pd.DataFrame() 
df["Sessions"]= metric
df["Range"]= dim
df= df[["Range","Sessions"]]

print(df)

In [ ]:
df.head()